# Mnist in libtorch

In [1]:
#pragma cling add_library_path("/home/abhishek/Downloads/libtorch-cxx11-abi-shared-with-deps-1.8.1+cpu/libtorch/lib/")
#pragma cling add_include_path("/home/abhishek/Downloads/libtorch-cxx11-abi-shared-with-deps-1.8.1+cpu/libtorch/include")
#pragma cling add_include_path("/home/abhishek/Downloads/libtorch-cxx11-abi-shared-with-deps-1.8.1+cpu/libtorch/include/torch/csrc/api/include/")

In [2]:
#pragma cling load("/home/abhishek/Downloads/libtorch-cxx11-abi-shared-with-deps-1.8.1+cpu/libtorch/lib/libc10.so")
#pragma cling load("/home/abhishek/Downloads/libtorch-cxx11-abi-shared-with-deps-1.8.1+cpu/libtorch/lib/libgomp-75eea7e8.so.1")
#pragma cling load("/home/abhishek/Downloads/libtorch-cxx11-abi-shared-with-deps-1.8.1+cpu/libtorch/lib/libtorch.so")

In [3]:
#include <torch/torch.h>
#include <iostream>
#include <typeinfo>

In [4]:
struct Net : torch::nn::Module
{
    Net()
        : conv1(torch::nn::Conv2dOptions(1, 6, 3)),
          conv2(torch::nn::Conv2dOptions(6, 16, 3)),
          fc1(400, 120),
          fc2(120, 84), fc3(84, 10)
    {
        register_module("conv1", conv1);
        register_module("conv2", conv2);
        register_module("fc1", fc1);
        register_module("fc2", fc2);
        register_module("fc3", fc3);
    }

    torch::Tensor forward(torch::Tensor x)
    {
        x = torch::max_pool2d(torch::relu(conv1->forward(x)), 2);

        x = torch::max_pool2d(torch::relu(conv2->forward(x)), 2);

        x = x.view({-1, 400});

        x = torch::relu(fc1->forward(x));
        x = torch::relu(fc2->forward(x));

        x = fc3->forward(x);

        return torch::log_softmax(x, /*dim=*/1);
    }

    torch::nn::Conv2d conv1;
    torch::nn::Conv2d conv2;
    torch::nn::Linear fc1;
    torch::nn::Linear fc2;
    torch::nn::Linear fc3;
};

In [5]:
Net model;

In [6]:
template <typename DataLoader>
void train(
    Net &model,
    DataLoader &data_loader,
    torch::optim::Optimizer &optimizer,
    torch::Device &device)
{
    model.train();
    size_t batch_idx = 0;
    for (auto &batch : data_loader)
    {
        auto data = batch.data.to(device);
        auto targets = batch.target.to(device);

        optimizer.zero_grad();
        auto output = model.forward(data);

        auto loss = torch::nll_loss(output, targets);

        AT_ASSERT(!std::isnan(loss.template item<float>()));

        loss.backward();
        optimizer.step();

        if (++batch_idx % 1000 == 0)
        {
            std::printf(
                "Batch %ld | Loss: %.4f\n",
                batch_idx,
                loss.template item<float>());
        }
    }
}

In [11]:
template <typename DataLoader>
void test(
    Net& model,
    torch::Device device,
    DataLoader& data_loader,
    size_t dataset_size) {
  torch::NoGradGuard no_grad;
  model.eval();
  double test_loss = 0;
  int32_t correct = 0;
  for (const auto& batch : data_loader) {
    auto data = batch.data.to(device), targets = batch.target.to(device);
    auto output = model.forward(data);
    test_loss += torch::nll_loss(
                     output,
                     targets,
                     /*weight=*/{},
                     torch::Reduction::Sum)
                     .template item<float>();
    auto pred = output.argmax(1);
    correct += pred.eq(targets).sum().template item<int64_t>();
  }

  test_loss /= dataset_size;
  std::printf(
      "\nTest set: Average loss: %.4f | Accuracy: %.3f\n",
      test_loss,
      static_cast<double>(correct) / dataset_size);
}


In [12]:
Net model;

torch::DeviceType device_type;

if (torch::cuda::is_available())
{
    device_type = torch::kCUDA;
}
else
{
    device_type = torch::kCPU;
}
torch::Device device(device_type);

model.to(device);

auto train_dataset = torch::data::datasets::MNIST("/home/abhishek/Desktop/libtorch-example-codes/Mnist/data").map(torch::data::transforms::Stack<>());

auto train_loader = torch::data::make_data_loader(std::move(train_dataset), 4);

torch::optim::SGD optimizer(model.parameters(), torch::optim::SGDOptions(0.01).momentum(0.5));

auto test_dataset = torch::data::datasets::MNIST("/home/abhishek/Desktop/libtorch-example-codes/Mnist/data", torch::data::datasets::MNIST::Mode::kTest)
                          .map(torch::data::transforms::Stack<>());

const size_t test_dataset_size = test_dataset.size().value();
auto test_loader = torch::data::make_data_loader(std::move(test_dataset), 4);



In [9]:
train(model, *train_loader, optimizer, device);

Batch 1000 | Loss: 0.4825
Batch 2000 | Loss: 0.0558
Batch 3000 | Loss: 0.0112
Batch 4000 | Loss: 0.3857
Batch 5000 | Loss: 0.0433
Batch 6000 | Loss: 0.0313
Batch 7000 | Loss: 0.0032
Batch 8000 | Loss: 0.0001
Batch 9000 | Loss: 0.1220
Batch 10000 | Loss: 0.3579
Batch 11000 | Loss: 0.0418
Batch 12000 | Loss: 0.0486
Batch 13000 | Loss: 0.0011
Batch 14000 | Loss: 0.0085
Batch 15000 | Loss: 0.0041


In [14]:
// test(model, *test_loader, device, test_dataset_size)

> # test function needs some fixing